### [Cookie Cats](https://www.youtube-nocookie.com/embed/GaP5f0jVTWE) is a popular mobile game where players work to clear a board and advance by connecting three matching tiles 

#### The data you will be analyzing in this practice notebbok was retrieved from [kaggle](https://www.kaggle.com/yufengsui/mobile-games-ab-testing) but originated as part of a [DataCamp project](https://learn.datacamp.com/projects/184). The variables in the dataset are:
**userid** - a unique identifier for each player    
**version** - whether the player was put in the control group (gate_30) or the experimental group (gate_40)  
**sum_gamerounds** - the number of game rounds played by the player during the first 14 days after installation    
**retention_1** - did the player return to play the game 1 day after installing?  
**retention_7** - did the player return to play the game 7 days after installing?  


#### An A/B Test was designed and run to see if users would be more likely to continue playing Cookie Cats 7 days after installing it if they could play to level 40 before encountering a gate instead of hitting their first gate at level 30
 - Gates are used in the game to require a user to wait before continuing play _unless_ they make an in-app purchase. 
 - Two versions of the game were created. One presented the first gate at level 30 (Group A) and the other presented the first gate at level 40 (Group B)
 - Users were randomly assigned to Group A or Group B when they downloaded the app
 - A sample of 45,000 users in each group was determined to be sufficient to measure an effect

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from statsmodels.stats.proportion import proportions_ztest

#### Get the results of your experiment by loading `cookies_cats.csv`

In [ ]:
results = pd.read_csv('../data/cookie_cats.csv')
results.head()

In [ ]:
results.shape

In [ ]:
results.version.value_counts()

In [ ]:
[groupB_obs, groupA_obs] = results.version.value_counts().values

#### First we will calculate the 1-day retention rate and evaluate it for statistical significance together as a warmup exercise
 - find the number of players in each group who returned after 1 day
 - for the point estimates, calculate the proportion of all players in each group who returned by dividing the total number for each group by the count of the players who returned

In [ ]:
results.groupby('version')['retention_1'].agg('sum')

- The code in the cell above sums up the rows where the retention_1 is true for each version of the game, giving us the number of users in each group that returned to play the game 1 day after installing it
- In the next cell we'll save those values to an array 

In [ ]:
[groupA_1day_count, groupB_1day_count] = results.groupby('version')['retention_1'].agg('sum').values

In [ ]:
results.groupby('version')['retention_1'].agg('sum') / results.groupby('version')['retention_1'].count()

- In the cell above we calculate the proportion of users in each group who returned to the game after one day by dividing the number in each group who returned by the total number assigned to the respective group
- We'll save those proportions to an array below

In [ ]:
[groupA_est, groupB_est] = results.groupby('version')['retention_1'].agg('sum') / results.groupby('version')['retention_1'].count()

In [ ]:
print('Number of observations group A:', groupA_obs)
print('Number of observations group B:', groupB_obs)
print('Proportion retained for 1-day group A:', groupA_est)
print('Proportion retained for 1-day group B:', groupB_est)

#### We can use the retention estimates to determine the margin of error
 - First calculate the z-score:  
  ```z = norm.ppf(1 - (1 - conf_level) / 2)```   
  
 - Then calculate the margin of error for each point estimate:   
 ```margin_of_error = 100 * z * np.sqrt(est * (1 - est) / num_obs)```
    
where `est` is the point estimate and `num_obs` is the number of observations

In [ ]:
# using a 95% confidence interval
conf_level = 0.95


z = norm.ppf(1 - (1 - conf_level) / 2)


margin_of_error_a = z * np.sqrt(groupA_est * (1 - groupA_est) / groupA_obs)
margin_of_error_b = z * np.sqrt(groupB_est * (1 - groupB_est) / groupB_obs)

In [ ]:
print('The 95% Confidence Interval for 1-day retention for Group A is between\n',
      groupA_est - margin_of_error_a, 'and', groupA_est + margin_of_error_a)
print('')
print('The 95% Confidence Interval for 1-day retention for Group B is between\n',
      groupB_est - margin_of_error_b, 'and', groupB_est + margin_of_error_b)

#### You can use the `proportions_ztest()` method from *statsmodels.stats.proportion* to calculate the p-value

In [ ]:
test_statistic, p_value = proportions_ztest(count = [groupA_1day_count, groupB_1day_count], 
                  nobs = [groupA_obs, groupB_obs])

In [ ]:
p_value

#### Your turn: determine if the version of Cookie Cats that does not present a gate until level 40 retains more users by the 7th day than the version that presents a gate at level 30, and determine if the difference rate is statistically significant
**Using a markdown cell**    
1. State the null and alternative hypotheses
2. What is the single variant (difference) between your A and B groups?
3. How is random assignment to each group accomplished?
3. What is the target outcome to measure (your point statistic)?
4. Has a sufficient sample size been determined?    

**Using code cells**
1. Calculate the following for each group:
    a. number of observations
    b. number retained for 7 days
    c. proportion retained for 7 days
2. Determine the margin of error at the 95% confidence level.
3. Determine the p-value.

**Be prepared to share your opinion on whether you will accept or rejact the null hypothesis.**